In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive, fixed
from ipywidgets import IntSlider
import networkx as nx
import re

import sys
sys.path.append('..')
from transform.read_transform import read_transform

In [74]:
data = read_transform(path_tweets='../data/TW.csv', path_retweets='../data/RT.csv', 
                      join_method='concat', 
                      hashtags_as_list=False, mentions_as_list=False)
print(data.shape)

(44334, 19)


## Get all the mentions in a dictionary

In [82]:
tmp = data[['mentions', 'user_screen_name']]
tmp = tmp[tmp['mentions'].isnull() == False]
tmp.loc[:,'mentions'] = tmp.loc[:,'mentions'].str.replace("'", "").str.split(', ')
print(tmp.shape)

tmp = tmp.set_index('user_screen_name')
tmp.head(2)

(34171, 2)


,mentions
user_screen_name,
ReAnimetron,[byOceanEyes]
Jenna_Blum,"[AMIGHTYBLAZE, fascicles]"


In [83]:
mentions = tmp.to_dict()['mentions']
pd.Series(mentions)[:2]

ReAnimetron                [byOceanEyes]
Jenna_Blum     [AMIGHTYBLAZE, fascicles]
dtype: object

## Get all the attributes in a dictionary

In [84]:
tmp = data[['user_screen_name', 'followers_count']].set_index('user_screen_name')
follower_counts = tmp.to_dict()['followers_count']
pd.Series(follower_counts)[:2]

ReAnimetron       5064
DiningCooking    16020
dtype: int64

## Create Graph-File

In [98]:
G = nx.DiGraph()

In [99]:
# Add nodes
for key, mention in mentions.items():
    G.add_node(key, followers_count=follower_counts[key])
    if type(mentions) != float:
        for m in mentions:
            G.add_node(m, followers_count=follower_counts[m])

In [100]:
pd.Series(nx.get_node_attributes(G, 'followers_count'))[:5]

ReAnimetron     5064
Jenna_Blum      6090
tbevents_        822
DGSpeaks       10223
lealalicia      1035
dtype: int64

In [101]:
# Add edges
for key, val in mentions.items():
    if type(val) != float:
        for v in val:
            G.add_edge(key, v)

In [102]:
nx.write_gexf(G, path='./myData/mention_attr.gexf')